In [65]:
import os
dirName = "announcements"
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  announcements  already exists


In [63]:
from googlesearch import search 
query = "\"S&P 500 INDEX –\" \"Set to Join\" site:us.spindices.com filetype:pdf"
results = search(query, tld='com', lang='en', num=10, start=0, stop=None, pause=2.0)

In [62]:
for result in results:
    print(result)

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [ ]:
import requests
import os
import shutil

global dump

def download_file():
    global dump
    url = "http://randomsite.com/file.gz"
    file = requests.get(url, stream=True)
    dump = file.raw

def save_file():
    global dump
    location = os.path.abspath("D:\folder\file.gz")
    with open("file.gz", 'wb') as location:
        shutil.copyfileobj(dump, location)
    del dump